<a href="https://colab.research.google.com/github/LuHellerKP/SMA_PUCRS/blob/main/Entrega_%7C_Simulador_para_Rede_de_Filas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import random
import yaml
from collections import deque

class Fila:
    def __init__(self, id, servidores, capacidade, tempo_chegada=None, tempo_atendimento=None):
        self.id = id
        self.servidores = servidores
        self.capacidade = capacidade
        self.tempo_chegada = tempo_chegada
        self.tempo_atendimento = tempo_atendimento
        self.clientes = deque()
        self.servidores_ocupados = 0
        self.clientes_perdidos = 0

    def chegada(self, cliente, tempo_atual):
        if len(self.clientes) < self.capacidade:
            self.clientes.append((cliente, tempo_atual))
        else:
            self.clientes_perdidos += 1

    def atender(self):
        if self.servidores_ocupados < self.servidores and len(self.clientes) > 0:
            cliente, tempo_chegada = self.clientes.popleft()
            tempo_atendimento = random.uniform(*self.tempo_atendimento)
            self.servidores_ocupados += 1
            return tempo_atendimento
        return None

    def liberar_servidor(self):
        self.servidores_ocupados = max(0, self.servidores_ocupados - 1)

class SimuladorRedeFilas:
    def __init__(self, filas, roteamento):
        self.filas = {fila.id: fila for fila in filas}
        self.roteamento = roteamento
        self.tempo_simulacao = 0

    def simular_cliente(self, fila_atual, cliente):
        while True:
            fila_atual.liberar_servidor()
            tempo_atendimento = fila_atual.atender()

            if tempo_atendimento is None:
                break

            self.tempo_simulacao += tempo_atendimento

            proxima_fila = self.definir_proxima_fila(fila_atual.id)
            if proxima_fila == "saida":
                break
            else:
                self.filas[proxima_fila].chegada(cliente, self.tempo_simulacao)

    def definir_proxima_fila(self, fila_id):
        destino = random.uniform(0, 1)
        for r in self.roteamento[fila_id]:
            if destino <= r['probabilidade']:
                return r['destino']
        return "saida"

    def executar(self, num_clientes):
        for cliente in range(num_clientes):
            self.simular_cliente(self.filas[1], cliente)

    def relatorio(self):
        resultados = {}
        for fila in self.filas.values():
            resultados[fila.id] = {
                'clientes_perdidos': fila.clientes_perdidos,
                'tempo_acumulado': self.tempo_simulacao
            }
        return resultados

def carregar_configuracao(arquivo_yaml):
    with open(arquivo_yaml, 'r') as file:
        config = yaml.safe_load(file)
    filas = []
    roteamento = {}
    for fila_conf in config['filas']:
        fila = Fila(
            id=fila_conf['id'],
            servidores=fila_conf['servidores'],
            capacidade=fila_conf['capacidade'],
            tempo_chegada=fila_conf.get('tempo_chegada', None),
            tempo_atendimento=fila_conf['tempo_atendimento']
        )
        filas.append(fila)
        roteamento[fila_conf['id']] = fila_conf['roteamento']
    return filas, roteamento

# Carregar arquivo de configuração
filas, roteamento = carregar_configuracao('/rede_de_filas.yaml')

# Criar simulador
simulador = SimuladorRedeFilas(filas, roteamento)

# Executar simulação
simulador.executar(100000)

# Relatório final
resultados = simulador.relatorio()
print(resultados)


{1: {'clientes_perdidos': 0, 'tempo_acumulado': 0}, 2: {'clientes_perdidos': 0, 'tempo_acumulado': 0}, 3: {'clientes_perdidos': 0, 'tempo_acumulado': 0}}
